# Static Visualisations with Matplotlib

In [ ]:
import os
import geopandas as gpd
import contextily as cx
import geoplot.crs as gcrs
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import geoplot as gplt
import mapclassify as mc
import pandas as pd
from dotenv import load_dotenv, find_dotenv

In [ ]:
# makes the inline-figures in notebooks look crisper
%config InlineBackend.figure_format = 'svg'

In [ ]:
load_dotenv(find_dotenv(usecwd=True));

In [ ]:
data_folder=os.getenv('DATA_FOLDER')

## Visualising the Swiss cantons

In [ ]:
cantons = gpd.read_file(os.path.join(data_folder, 'swissboundaries', 'swissBOUNDARIES3D_1_4_TLM_KANTONSGEBIET.shp'))

In [ ]:
cantons.columns

In [ ]:
cantons.head()

**To do**: limit the content of the tooltips!

In [ ]:
cantons.explore()

## Pointplots

**To Do:** 
* Scale the markersize by population number
* add a legend to the plot
* annotate each point with the canton name

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))

cantons.boundary.plot(ax=ax, color='grey', linewidth=0.6, alpha=0.5)

cantons[~cantons['EINWOHNERZ'].isna()].centroid.plot(ax=ax, marker='o', color='tab:blue', markersize=20)

ax.spines[:].set_visible(False)


ax.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)

plt.show()

## From Pointplots to density plots

In [ ]:
boroughs = gpd.read_file(gplt.datasets.get_path("nyc_boroughs"))
collisions = gpd.read_file(gplt.datasets.get_path("nyc_collision_factors"))

In [ ]:
boroughs

In [ ]:
boroughs.crs

In [ ]:
collisions

In [ ]:
collisions.explore()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

# boroughs.to_crs(epsg=3857).plot(ax=ax, cmap='Pastel1', alpha=0.8)

collisions.to_crs(epsg=3857).plot(ax=ax, marker='o', color='tab:blue', markersize=10)

# cx.add_basemap(ax)

ax.spines[:].set_visible(False)


ax.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)

plt.show()

In [ ]:
collisions.crs

In [ ]:
ax = gplt.kdeplot(
    collisions.head(1000), 
    # clip=boroughs,
     cmap='Reds', fill=True, alpha=0.8
)

cx.add_basemap(ax, crs=collisions.crs) # 'EPSG:5070'

# ax = gplt.polyplot(boroughs, projection=gcrs.AlbersEqualArea())
# gplt.kdeplot(collisions, cmap="Reds", fill=True, clip=boroughs, ax=ax)
plt.show()

## Choropleth maps

In [ ]:
cantons['EWZ_REL'] = cantons['EINWOHNERZ']/cantons.area

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))

cantons.boundary.plot(ax=ax, color='grey', linewidth=0.6)
cantons.plot(ax=ax, column='EWZ_REL', cmap='RdBu_r', 
             norm=mpl.colors.LogNorm(vmin=cantons.EWZ_REL.min(), vmax=cantons.EWZ_REL.max()),
             legend=True,
            legend_kwds={"label": "Population in 2014 relative to area", "orientation": "vertical",
                        'location': 'right', 'shrink': 0.5}
            )

ax.spines[:].set_visible(False)


ax.tick_params(left = False, right = False , labelleft = False ,
                labelbottom = False, bottom = False)

plt.show()

## The Big Mac-price on a worldmap
**To do**:
* Merge the data frame with the big mac prizes with the geo dataframe of the world's countries boundaries
* Create a choropleth map coloring each country by big mac price

In [ ]:
df_mac = pd.read_csv(os.path.join(data_folder, 'bigmac', 'big_mac_2020.csv'))

In [ ]:
worldne = gpd.read_file(os.path.join(data_folder, 'natural_earth', '110m_cultural', 'ne_110m_admin_0_countries.shp'))
for c in worldne.columns:
    print(c)

In [ ]:
worldne[['POP_EST', 'CONTINENT', 'NAME', 'ISO_A3', 'GDP_MD', 'geometry']]

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
world

In [ ]:
world_merged = ...

## Cartograms

In [ ]:
contiguous_usa = gpd.read_file (gplt.datasets.get_path ('contiguous_usa'))

In [ ]:
scheme = mc.Quantiles(contiguous_usa['population'], k=5)

In [ ]:
ax = gplt.cartogram(
    contiguous_usa, scale='population', projection=gcrs.AlbersEqualArea(),
    legend=True, legend_kwargs={'bbox_to_anchor': (1, 0.9)}, legend_var='hue',
    hue='population', scheme=scheme, cmap='Greens',
    legend_labels=[
        '<1.4 million', '1.4-3.2 million', '3.2-5.6 million',
        '5.6-9 million', '9-37 million'
    ],
    limits=(0.5, 1)
)
gplt.polyplot(contiguous_usa, facecolor='lightgray', edgecolor='white', ax=ax)

plt.show()